### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch
import nd2
import numpy as np

from skimage.transform import rescale

import utils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:

x = tifffile.imread("Data/01-yeast_many.tif")
x.shape

(2048, 2048)

Add the image to the napari viewer

In [4]:
viewer.layers.clear()
viewer.add_image(x)

<Image layer 'x' at 0x17346cfa0>

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [5]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with flows and probabilities
- styles
- diams

#### Try cellpose with no paramter changes

In [6]:

masks, flows, styles, diams = model.eval(x, channels=[0, 0])
diams

30.0

Show the results from cellpose a layers in napari

In [7]:
viewer.layers.clear()
viewer.add_image(x)
viewer.add_labels(masks)


<Labels layer 'masks' at 0x2f03203d0>

See what happens when the diameter is changed. What diameter is the best?

In [9]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[2], visible=False, name="probability")

<Image layer 'probability' at 0x3220bcf40>